(comp)=
# Comparación de algoritmos
El rendimiento de los algoritmos utilizados anteriormente no puede ser comparado directamente por las ditribuciones de las variables de acuerdo a la clasificación realizada por los modelos. Como se evidencia en la sección anterior, un algoritmo puede parecer ajustar muy bien a los datos en una variable, pero no en otra. Por ello, utilizamos las métricas explicadas en la {numref}`met` para comparar directamente el rendimiento de los algoritmos. En esta sección se compararán los algoritmos utilizados en la sección anterior, agregando UCluster y GAN-AE. 

Los resultados presentados en esta sección se obtuvieron utilizando el pipeline de `benchtools`, descrito en la {numref}`bench-pipeline-cap`.

(comp-RnD)=
## Conjunto R&D
En la sección anterior, se mostraron las distribuciones de algunas variables de acuerdo a la clasificación realizada por cada modelo. Esto se hizo con el objetivo de tener una idea general de cómo están clasificando los algoritmos y para poder asociar esta clasificación a las métricas que se mostrarán a continuación.

La clasificación realizada por los algoritmos de las LHCO 2020 para el conjunto R&D no se lleva a cabo utilizando el mismo subconjunto de datos en todos los modelos. Los subconjuntos de entrenamiento y prueba varían para los algoritmos de las olimpiadas de acuerdo al código publicado por los participantes. Sin embargo, el subconjunto para utilizado para calcular las métricas a continuación no fue utilizado para entrenamiento en ninguno de los modelos.
### Métricas numéricas
En la {numref}`comp-metricas-num-RnD` se observa una comparación visual de las métricas numéricas: *exactitud balanceada*, *precisión*, *recuperación* y *puntaje f1*.

In [6]:
from benchtools.src.plotools import image_grid

# Definimos el path de las imagenes
PATH_FIGURES = '../../figuras/'

In [7]:
lista_imagenes = ['../../figuras/comp-balancedacc-RnD.png','../../figuras/comp-precision-RnD.png',
                '../../figuras/comp-recall-RnD.png','../../figuras/comp-f1score-RnD.png']
image_grid(rows=2, columns=2, images=lista_imagenes, name='comp-metricas-num-RnD', path=PATH_FIGURES, remove=False)

```{figure} ../../figuras/comp-metricas-num-RnD.png
---
width: 80%
name: comp-metricas-num-RnD
---
Diagramas de barras de las métricas numéricas obtenidas utilizando el pipeline de `benchtools` para el conjunto R&D. De izquierda a derecha, en la fila superior: exactitud balanceada y precisión. En la fila inferior: recuperación y puntaje f1.
```
Todos los modelos obtuvieron un puntaje de exactitud balanceada mayor a 70%, lo que se considera alto. Sin embargo, observamos que la precisión y la recuperación varía entre los clasificadores. 

Los modelos supervisados QDA, GB y RFC obtuvieron altos puntajes de precisión pero bajos puntajes de recuperación. Esto significa que los algoritmos clasifican pocos eventos como señal, pero estas clasificaciones son mayormente correctas. En contraste, el modelo MLP obtuvo un alto puntaje de precisión y recuperación. A su vez, el modelo de TensorFlow obtuvo un menor puntaje de precisión pero el mayor puntaje de recuperación. Es decir, clasifica más eventos como señal, clasificando eventos de fondo incorrectamente, pero logra clasificar la mayoría de los eventos de señal con la etiqueta correcta.

Los modelos no supervisados KMeans, UCluster y GAN-AE son los modelos con menor precisión pero mayor recuperación. Es decir, están etiquetando una mayor cantidad de eventos de fondo incorrectamente como señal, pero entre los eventos etiquetados como señal, se encuentra la mayor parte de los eventos de señal real. 

La media armónica de estas dos métricas se observa en el gráfico del puntaje f1, donde, en general, los modelos supervisados obtuvieron puntajes más altos que los modelos no supervisados. 

El mayor puntaje f1 fue alcanzado por MLP. Por otra parte, el mayor puntaje de precisión lo logró RFC y el mayor puntaje de recuperación lo consiguió el modelo de TensorFlow, que también obtuvo el mayor puntaje de exactitud balanceada. Los menores puntajes fueron obtenidos por UCluster, con la menor precisión y menor puntaje f1, y por GBC, con el menor puntaje de exactitud balanceada y recuperación. 

Los modelos supervisados obtienen una mejor clasificación porque poseen más información en el momento del aprendizaje, puesto que utilizan las etiquetas de los eventos, mientras que los modelos no supervisados requieren estructura en los datos de entrenamiento. Si el conjunto de datos original no posee esta estructura naturalmente, se debería lograr mediante el pre-procesamiento. Sin embargo, el pre-procesamiento de los tres modelos no-supervisados no logró una estructura suficientemente significativa como para obtener una mejor clasificación que las de los modelos supervisados.

Se consideró una clasificación aleatoria en la que un modelo predice que el 50% de los eventos son señal y el 50% son fondo, aleatoriamente. La clasificación aleatoria obtuvo los menores puntajes excepto en la precisión y el puntaje f1, donde UCluster obtuvo menor puntaje. En general, los modelos clasifican mejor que un clasificador aleatorio, salvo UCluster.

Un resumen de los valores numéricos obtenidos se encuentran en la {numref}`comp-metricasnumericas-RnD`.

```{table} Métricas numéricas obtenidas con el pipeline de *benchtools* para el conjunto R&D.
:name: comp-metricasnumericas-RnD

| Classifier                    |   Balanced accuracy |   Precision |   Recall |   F1 score |
|:-----------------------------:|:-------------------:|:-----------:|:--------:|:----------:|
| Random classification         |            0.4991   |   0.0906    |  0.1534  |  0.4984    |
| TensorflowClassifier          |            0.9076   |   0.5033    |  0.9044  |  0.6467    |
| RandomForestClassifier        |            0.8127   |   0.8556    |  0.6362  |  0.7298    |
| GradientBoostingClassifier    |            0.7930   |   0.8145    |  0.5997  |  0.6908    |
| QuadraticDiscriminantAnalysis |            0.8146   |   0.6973    |  0.6578  |  0.6770    |
| MLPClassifier                 |            0.8623   |   0.8391    |  0.7388  |  0.7858    |
| KMeans                        |            0.7934   |   0.2551    |  0.8288  |  0.3901    |
| GAN-AE                        |            0.7950   |   0.3962    |  0.8431  |  0.5391    |
| UCluster                      |            0.8073   |   0.0308    |  0.8921  |  0.0595    |

```
### Métricas bidimensionales
En la sección anterior se consideró la clasificación después de cortar en un umbral específico para cada clasificador. A continuación, observaremos las métricas bidimensionales, que proporcionan información más general del rendimiento de los algoritmos, porque se obtiene el rendimiento para múltiples umbrales.

Las métricas bidimensionales obtenidas con el pipeline de `benchtools` son la *eficiencia de señal vs. rechazo de fondo*, la *ROC inversa*, la *curva precisión recuperación* y la *mejora de la significancia*.

In [8]:
lista_imagenes = ['../../figuras/comp-rejection-RnD.png','../../figuras/comp-inverseroc-RnD.png',
                '../../figuras/comp-precisionrecall-RnD.png','../../figuras/comp-significance-RnD.png']
image_grid(rows=2, columns=2, images=lista_imagenes, name='comp-metricas-graf-RnD', path=PATH_FIGURES, remove=False)

De acuerdo a todas las métricas bidimensionales, el modelo MLP es el que tiene un mejor rendimiento al clasificar el conjunto R&D, con un AUC de 0.975 y una precisión promedio de 0.862, como se observa en la {numref}`comp-metricas-graf-RnD`.
```{figure} ../../figuras/comp-metricas-graf-RnD.png
---
width: 90%
name: comp-metricas-graf-RnD
---
Metricas bidimensionales para la clasificación del conjunto R&D. De izquierda derecha, en la fila superior: eficiencia de señal vs. rechazo de fondo, ROC inversa, y en la fila inferior: precisión-recuperación y mejora significativa.
```
Del gráfico de eficiencia de señal vs. rechazo de fondo y del gráfico de precisión-recuperación, observamos que el clasificador de TensorFlow se encuentra en segundo lugar con un AUC de 0.969 y una precisión media de 0.832. Seguido de estos modelos, se encuentran RFC, GBC y QDA en todas las métricas.

Los modelos no supervisados obtuvieron menores puntajes que los clasificados. De acuerdo al AUC, GAN-AE obtiene un mejor resultado que UCluster, con 0.866 y 0.865, respectivamente. GAN-AE también obtiene un mejor resultado según la curva de precisión-recuperación, con una precisión media de 0.521, mientras que la clasificación de UCluster da una precisión media de 0.060.

Por último, KMeans obtuvo el menor AUC y la curva de este modelo se encuentra por debajo de todos los demás modelos en todos los gráficos, excepto el de precisión-recuperación. De acuerdo a este gráfico, KMeans obtiene un mejor resultado que UCluster, con una precisión promedio de 0.130 y 0.060, respectivamente.

En este trabajo, es de interés conocer qué tan bien funcionan los algoritmos al clasificar señal y fondo en conjuntos de datos desconocidos para los modelos, con variaciones en las distribuciones de masa, como el conjunto BB1.

(comp-BB1)=
## Conjunto BB1
A diferencia de las predicciones realizadas con el cojunto R&D, las predicciones para el conjunto BB1 se hacen sobre todos los eventos, utilizando los modelos entrenados previamente con el 70% del conjunto R&D. Las métricas obtenidas en este apartado proveen una medida más cercana al verdadero rendimiento de los algoritmos para detectar anomalías.

### Métricas numéricas
En la {numref}`comp-metricas-num-BB1` vemos que las métricas varían con respecto a lo obtenido para el conjunto R&D. 

In [9]:
lista_imagenes = ['../../figuras/comp-balancedacc-BB1.png','../../figuras/comp-precision-BB1.png',
                '../../figuras/comp-recall-BB1.png','../../figuras/comp-f1score-BB1.png']
image_grid(rows=2, columns=2, images=lista_imagenes, name='comp-metricas-num-BB1', path=PATH_FIGURES, remove=False)

```{figure} ../../figuras/comp-metricas-num-BB1.png
---
width: 100%
name: comp-metricas-num-BB1
---
Diagramas de barras de las métricas numéricas obtenidas utilizando el pipeline de `benchtools` para el conjunto BB1. De izquierda a derecha, en la fila superior: exactitud balanceada y precisión. En la fila inferior: recuperación y puntaje f1.
```
La exactitud balanceada sigue siendo alta para todos los algoritmos. La precisión disminuyó notablemente para todos los modelos. En este conjunto, la mayor precisión fue lograda por RFC con un 3% de precisión, contrario a la precisión obtenida para el conjunto R&D, de 86%. Esto implica que los algoritmos clasifican más eventos de fondo incorrectamente, o menos eventos de señal correctamente, en el conjunto BB1.

La recuperación aumentó para los modelos GBC, GAN-AE y UCluster. Para UCluster, GAN-AE, KMeas y el clasificador de TensorFlow, la recuperación se mantuvo cercana o sobre el 80%. La baja precisión y alta recuperación indica que los modelos están clasificando una gran cantidad de eventos de fondo como señal. Sin embargo, en estos eventos clasificados como señal, logran clasificar la señal real.

Debido a la baja precisión, el puntaje f1 se vio afectado y disminuyó notablemente para todos los clasificadores. El clasificador con mayor puntaje es RFC, con un puntaje f1 de 0.05, mientras que en el conjunto R&D obtuvo 0.73. 

Nuevamente, el clasificador aleatorio obtuvo métricas más bajas que todos los modelos, excepto en la precisión y el puntaje f1, donde UCluster obtuvo las métricas más bajas. 

```{table} Métricas numéricas obtenidas con el pipeline de *benchtools* para el conjunto BB1.
:name: comp-metricasnumericas-BB1

| Classifier                    |   Balanced accuracy |   Precision |    F1 score |   Recall |
|:-----------------------------:|:-------------------:|:-----------:|:-----------:|:--------:|
| Random classification         |            0.4994   |    0.0008   |    0.0017   |   0.4988 |
| TensorflowClassifier          |            0.8418   |    0.0049   |    0.0096   |   0.8249 |
| RandomForestClassifier        |            0.7005   |    0.0362   |    0.0666   |   0.4101 |
| GradientBoostingClassifier    |            0.7164   |    0.0367   |    0.0678   |   0.4424 |
| QuadraticDiscriminantAnalysis |            0.6894   |    0.0264   |    0.0495   |   0.3909 |
| MLPClassifier                 |            0.8143   |    0.0028   |    0.0055   |   0.9005 |
| KMeans                        |            0.8094   |    0.0039   |    0.0078   |   0.7854 |
| GAN-AE                        |            0.9091   |    0.0201   |    0.0393   |   0.8752 |
| UCluster                      |            0.8070   |    0.0002   |    0.0003   |   0.9767 |
```
### Métricas bidimensionales
Las métricas bidimensionales también varían con respecto a las obtenidas utilizando el conjunto R&D. Los resultados se observan en la {numref}`comp-metricas-graf-BB1`.

In [10]:
lista_imagenes = ['../../figuras/comp-rejection-BB1.png','../../figuras/comp-inverseroc-BB1.png',
                '../../figuras/comp-precisionrecall-BB1.png','../../figuras/comp-significance-BB1.png']
image_grid(rows=2, columns=2, images=lista_imagenes, name='comp-metricas-graf-BB1', path=PATH_FIGURES, remove=False)

```{figure} ../../figuras/comp-metricas-graf-BB1.png
---
width: 90%
name: comp-metricas-graf-BB1
---
Metricas bidimensionales obtenidas para el conjunto BB1. De izquierda derecha, en la fila superior: eficiencia de señal vs. rechazo de fondo, ROC inversa, y en la fila inferior: precisión-recuperación y mejora significativa.
```
En todos los gráficos, es evidente que GAN-AE tiene un mejor rendimiento en este conjunto de datos. Asimismo, los clasificadores con menor rendimiento son KMeans y UCluster. Que un modelo no-supervisado haya logrado una mejor clasificación en este conjunto indica que tiene una mejor capacidad de generalización. Esta capacidad de generalización no se observó en los modelos supervisados porque se utilizaron variables depedientes de la masa, que además forman parte de las variables más importantes para la clasificación.

De acuerdo al gráfico de eficiencia de señal vs. rechazo de fondo, el algoritmo con mejores resultados después de GAN-AE es MLP, con un AUC de 0.953. Seguido a estos modelos, no es evidente cuál da un mejor resultado entre GBC y RFC y entre QDA y el clasificador de TensorFlow. De acuerdo al AUC, a MLP le sigue GBC, RFC, el clasificador de TensorFlow, QDA y RFC. Los algoritmos no supervisados KMeans y UCluster son los que tuvieron menor rendimiento, donde UCluster obtuvo el rendimiento más bajo.

De la métrica de precisión-recuperación también se obtuvo un mejor resultado para GAN-AE, seguido de GBC, el clasificador de TensorFlow y QDA, que obtuviero una mayor precisión promedio que el modelo MLP. A este le sigue RFC y por último los modelos no-supervisados KMeans y UCluster, donde UCluster obtuvo la menor presición promedio, de 0.000. Este mismo orden de redimiento de los modelos se obtiene en el gráfico de mejora significativa.

En general, las curvas de UCluster y KMeans se mantienen cercanas a las de un clasificador aleatorio, excepto en el gráfico de eficiencia de señal vs. rechazo de fondo, donde se obtienen curvas que indican una clasificación mejor a la del clasificador aleatorio.